In [1]:
import numpy as np
import rpy2
import rpy2.robjects as robjects
import pickle
from time import time

In [2]:
def store_results_dict(results, name):
    pickle.dump(results, open("./Results/{}.p".format(name), "wb" ))
    
def retrieve_results_dict(name):
    try:
        return pickle.load(open( "./Results/{}.p".format(name), "rb" ))
    except Exception as e:
        return None

### Data Generation

#### Raw Data

In [3]:
# GLOBAL CONFIG

# Var count
n_vars = 10

# Data types (default is standard normal)
binary_indeces = [1, 3, 6, 8, 9]
binarize = True

# Associations between vars an treat/outcome
treat_vars = [0,1,2,3,4,5,6,7]
outcome_vars = [0,1,2,3,4,8,9,10]

# Treat/outcome generation weights
assignment_weights = np.array([0, 0.8, -0.25, 0.6, -0.4, -0.8, -0.5, 0.7])
outcome_weights = np.array([-3.85, 0.3, -0.36, -0.73, -0.2, 0.71, -0.19, 0.26])
true_treat_effect = -0.4

def generate_data(n_samples=1000):
    # Generate 10 Random Vars
    # 1-4 are confounders: associated with outcome + treatment
    # 5-7 are exposure predictors
    # 8-10 are outcome predictors
    X = np.random.normal(loc=0.0, scale=1.0, size=(n_samples, n_vars))

    # Binarize specified vars if requested.
    if binarize:
        for var in binary_indeces:
            X[:, var-1] = (X[:, var -1] > 0).astype(int)

    # Add dummy for bias param     
    X = np.hstack([np.ones((n_samples, 1)), X])
    return X

In [4]:
# DEBUG
# X = generate_data(2000)
# X.shape

#### Assignment

In [5]:
# Create the models

assignment_models={}

def nonlinear_transform(X, B, quad_indeces):
    for quad_index in quad_indeces:
        quad = X[:, quad_index]**2
        X = np.hstack([X, quad.reshape(-1, 1)])
        B = np.append(B, B[quad_index])
    
    return X, B

def nonadditive_transform(X, B, interaction_indeces, interaction_weights=None):
    for interaction_index, var_indeces in enumerate(interaction_indeces):
        int_1, int_2 = var_indeces
        interaction_val = X[:, int_1]*X[:, int_2]
        
        if not interaction_weights:
            interaction_val = interaction_val*0.5
        else:
            interaction_val = interaction_val*interaction_weights[interaction_index]
            
        X = np.hstack([X, interaction_val.reshape(-1, 1)])
        B = np.append(B, B[int_1])
    
    return X, B

# Scenario 1
assignment_models["A_add_lin"] = lambda B, X: np.dot(X, B)

# Scenario 2:     
assignment_models["B_add_mild_nlin"] = lambda B, X: np.dot(*nonlinear_transform(X, B,
                                                       quad_indeces=[2]))
# Scenario 3:
assignment_models["C_add_mod_nlin"] = lambda B, X: np.dot(*nonlinear_transform(X, B,
                                                       quad_indeces=[2, 4, 7]))
# Scenario 4:
assignment_models["D_mild_nadd_lin"] = lambda B, X: np.dot(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (4,5), (5,6)]))

# Scenario 5:
assignment_models["E_mild_nadd_mild_nlin"] = lambda B, X: np.dot(*nonlinear_transform(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (4,5), (5,6)]), quad_indeces=[2]))
# Scenario 6
assignment_models["F_mod_nadd_lin"] = lambda B, X: np.dot(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (3,5), (4,6), (5,7), (1,6), (2,3),
                                                                            (3,4), (4,5), (5,6)],
                                                       interaction_weights=[0.5, 0.7, 0.5, 0.7, 0.5, 0.5, 0.7, 0.5, 0.5, 0.5]))
# Scenario 7
assignment_models["G_mod_nadd_mod_nlin"] = lambda B, X: np.dot(*nonlinear_transform(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (3,5), (4,6), (5,7), (1,6), (2,3),
                                                                            (3,4), (4,5), (5,6)],
                                                       interaction_weights=[0.5, 0.7, 0.5, 0.7, 0.5, 0.5, 0.7, 0.5, 0.5, 0.5]), 
                                                                            quad_indeces=[2, 4, 7]))

In [6]:
# Tests 
assert(set(assignment_models["A_add_lin"](np.array([2, 0.5, 1.5]),
                                                np.array([[1, 2,4], [1, 10, 20]]))) == set([9, 37]))

assert(set(assignment_models["B_add_mild_nlin"](np.array([2, 0.5, 1.5]),
                                                np.array([[1, 2,4], [1, 10, 20]]))) == set([33, 637]))

assert(set(assignment_models["C_add_mod_nlin"](np.array([2, 0.5, 1.5, 1, 1, 1, 2, 3]),
                                                np.array([[1, 2,4,5,6,7,8,9], [1, 10, 20, 30, 40, 50, 60, 60]]))) == set([373, 13457]))

assert(set(assignment_models["D_mild_nadd_lin"](np.array([2, 0.5, 1.5, 1, 1, 1, 2, 3]),
                                                np.array([[1, 2,4,5,6,7,8,9], [1, 10, 20, 30, 40, 50, 60, 60]]))) == set([139.5, 3632]))

assert(set(assignment_models["E_mild_nadd_mild_nlin"](np.array([2, 0.5, 1.5, 1, 1, 1, 2, 3]),
                                                np.array([[1, 2,4,5,6,7,8,9], [1, 10, 20, 30, 40, 50, 60, 60]]))) == set([163.5, 4232]))


#### Outcome and Assignment Functions

In [7]:
def get_assignments(B, X, n_samples, scenario="A_add_lin"):
    X_usable = X[:, treat_vars]
    
    # Calculate the probabilities of assignment
    linear_assignment_data = assignment_models[scenario](B, X_usable)
    p_treat = 1.0/(1+np.exp(-1*linear_assignment_data))

    # Assign
    rand = np.random.random(n_samples)
    assignments = (rand < p_treat).astype(int)
    
    return assignments

In [8]:
def get_outcomes(B, X, assignments, effect=true_treat_effect):
    X_usable = X[:, outcome_vars]
    return effect*assignments + np.dot(X_usable, B)

In [9]:
# DEBUG
# assignments = get_assignments(assignment_weights, X, "mild_nonaddititive_mild_nonlinear")
# outcomes = get_outcomes(outcome_weights, X, assignments)

### Analysis

In [10]:
from rpy2.robjects import IntVector, FloatVector, Formula
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri
numpy2ri.activate()

stats = importr('stats')
matching = importr('Matching')
snow = importr('snow')

Some code is going to >48 hours to ran. Lucky it's highly parellalisable so we can use a compute cluster. The one option is local to split across CPU cores. The better option is to go remote and explote 32 cores on multiple AWS machines.

On AWS, this is straightforward. Manually you need to port forward!. This allows the remote machine to connect to ports on the master via it's localhost loopback. 

```
# ~/.bash_profile
# Allow remote host to connect to local machine
# usage: $ remote_pfwd hostname {6000..6009}
function remote_pfwd {
  for i in ${@:2}
  do
    ssh -N -R $i:localhost:$i $1 &
  done
}
```
`remote_pfwd ubuntu@52.90.20.45 {11305..11307}`

In [11]:
AWS_MASTER_DNS="ip-172-31-42-147.ec2.internal"
AWS_SLAVE_1 = "ubuntu@ip-172-31-43-193.ec2.internal"
AWS_SLAVE_2 = "ubuntu@ip-172-31-81-244.ec2.internal"
AWS_MASTER_PORT_RANGE = list(range(11305, 11340))

class ClusterProvider(object):
    def __init__(self, n_nodes=8, remote_hosts=None, ports=None):
        if remote_hosts is None:
            self.cl = snow.makeSOCKcluster(["localhost"]*n_nodes)
        else:
            # Set the acceptable ports for connection
            # from the slaves
            if not ports:
                ports = AWS_MASTER_PORT_RANGE
            
            # Construct the connection string
            addresses = []
            for remote_host, n_nodes in remote_hosts:
                addresses+=[remote_host]*n_nodes
                
            self.cl = snow.makeSOCKcluster(addresses, rscript="Rscript", manual=False, snowlib="/usr/local/lib/R/site-library",
                                           port=IntVector(ports), master=AWS_MASTER_DNS, outfile="/dev/stdout", timeout=10)
    
    def get_cluster(self):
        return self.cl
    
    def kill_cluster(self):
        snow.stopCluster(self.cl)

In [12]:
# Local cluster
cluster_provider = ClusterProvider(n_nodes=14)

In [63]:
# Remote cluster
# cluster_provider = ClusterProvider(remote_hosts=[(AWS_SLAVE_1, 8)],
#                                     ports = list(range(11305, 11314)))

In [62]:
# Run this with True to kill the cluster
kill = True
if kill:
    cluster_provider.kill_cluster()

#### Estimators

Define methods which can process outcomes, assignments and covariate data into a treatment effect estimate. 

1. Logistic Regression
2. GenMatch
3. VAE

In [13]:
def get_propensity_scores(assignments, covariate_data):
    # Setup
    y = IntVector(assignments)
    fmla = Formula('y ~ X')
    env = fmla.environment
    
    # Run propensiy regression
    env['X'] = covariate_data
    env['y'] = y
    fit = stats.glm(fmla, family="binomial")
    
    # DEBUG: fit.rx("coefficients")
    return fit.rx2("fitted.values")

In [14]:
# 1. Logisic Regression Propensity Matching
def logistic_prop_matching_est(outcomes, assignments, covariate_data):
    
    propensity_scores = get_propensity_scores(assignments, covariate_data)
    
    # Run matching
    match_out = matching.Match(
        Y=FloatVector(outcomes),
        Tr=IntVector(assignments),
        X=propensity_scores,
        replace=True)
    
    return np.array(match_out.rx2("est").rx(1,1))[0]

In [15]:
# 2. GenMatch Matching
def genmatch_est(outcomes, assignments, covariate_data):
    
    # Get the singleton cluster
    cl = cluster_provider.get_cluster()
    
    # Add prop scores to covar data
    propensity_scores = np.array(get_propensity_scores(assignments, covariate_data))
    matching_data = np.hstack([covariate_data, propensity_scores.reshape(-1, 1)])
    
    start = time()
    gen_out = matching.GenMatch(
        Tr=IntVector(assignments),
        X=matching_data,
        BalanceMatrix=covariate_data,
        print_level=0,
        cluster=cl)
    print("GenMatch Time: ", time() - start)
    
    match_out = matching.Match(
        Y=FloatVector(outcomes),
        Tr=IntVector(assignments),
        X=matching_data,
        replace=True,
        Weight_matrix=gen_out)
    
    return np.array(match_out.rx2("est").rx(1,1))[0]

In [16]:
# DEBUG
# est = logistic_prop_matching_est(assignments, X[:, 1:]) # exclude the bias term
# np.array(est)

In [17]:
# DEBUG
# est = genmatch_est(assignments, X[:, 1:]) # exclude the bias term
# np.array(est)

#### Monte Carlo Runner Code

In [18]:
def get_data(n_samples, assignment_model):
    X = generate_data(n_samples)
    assignments = get_assignments(assignment_weights, X,
                                  n_samples, assignment_model)

    outcomes = get_outcomes(outcome_weights, X, assignments)
    
    return assignments, outcomes, X

def get_estimate(outcomes, assignments, covar_data, method, **args):
    return method(outcomes, assignments, covar_data, **args)

In [19]:
def run_simulation(runs=1000, n_samples=1000,
                   assignment_model="additive_linear",
                   estimator=logistic_prop_matching_est,
                   verbose=True,
                   **args):
    
    progress_tick = max(1, int(runs/10))
    results = np.zeros(runs)

    for i in range(runs):
        assignments, outcomes, covar_data = get_data(n_samples, assignment_model)
        
        covar_data = covar_data[:, 1:] #exclude bias term
        results[i] = get_estimate(outcomes, assignments,
                                  covar_data, estimator, **args)
        
        if i%progress_tick == progress_tick-1 and verbose:
            print("Done {} of {}".format(i+1, runs))
    
    bias = np.abs(np.mean((true_treat_effect-results)/true_treat_effect)*100)
    rmse = np.mean((true_treat_effect-results)**2)**0.5
    
    if verbose:
        print("\nRMSE", rmse)
        print("Bias", bias)
        print("===============\n\n")
    
    return {"RMSE": rmse, "Bias": bias}

In [20]:
# DEBUG:
run_simulation(runs=20, n_samples=1000, assignment_model="D_mild_nadd_lin",
              estimator=genmatch_est, verbose=True)

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required namespace: rgenoud

  warnings.warn(x, RRuntimeWarning)
/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required namespace: parallel

  warnings.warn(x, RRuntimeWarning)


GenMatch Time:  11.083404302597046
GenMatch Time:  31.12960457801819
Done 2 of 20
GenMatch Time:  11.981449365615845
GenMatch Time:  16.15269660949707
Done 4 of 20
GenMatch Time:  14.55371356010437
GenMatch Time:  13.442266702651978
Done 6 of 20
GenMatch Time:  14.062756061553955
GenMatch Time:  15.857194662094116
Done 8 of 20
GenMatch Time:  8.22305178642273
GenMatch Time:  12.44659686088562
Done 10 of 20
GenMatch Time:  10.008002996444702
GenMatch Time:  13.095482110977173
Done 12 of 20
GenMatch Time:  11.391975164413452
GenMatch Time:  15.019485712051392
Done 14 of 20
GenMatch Time:  10.63561201095581
GenMatch Time:  14.826309204101562
Done 16 of 20
GenMatch Time:  13.209826946258545
GenMatch Time:  12.118821620941162
Done 18 of 20
GenMatch Time:  17.138674020767212
GenMatch Time:  10.279953002929688
Done 20 of 20

RMSE 0.043918225298051665
Bias 0.3755681319660123




{'Bias': 0.3755681319660123, 'RMSE': 0.043918225298051665}

### Run MC trial

In [21]:
assignment_model_names = ['A_add_lin', 'B_add_mild_nlin', 'C_add_mod_nlin', 'D_mild_nadd_lin',
                     'E_mild_nadd_mild_nlin', 'F_mod_nadd_lin', 'G_mod_nadd_mod_nlin']

def get_store_name(models_being_run, est, runs, n_samples):
    # Storage
    
    if set(models_being_run) == set(assignment_model_names):
        store_name = "est_{}_runs_{}_n_{}".format(
            est.__name__,
            runs,
            n_samples)
    else:
        store_name = "est_{}_runs_{}_n_{}_models_{}".format(
            est.__name__,
            runs,
            n_samples,
            "_".join(models_being_run))
    
    return store_name

def run_test_battery(est,
                     store_name=None, 
                     runs=1000,
                     n_samples=1000,
                     models=assignment_models,
                     overwrite=False, verbosity=1,
                     **args):
    # Logging
    def printer(level, *args):
        if level <= verbosity:
            print(*args)
    
    # Storage config
    if store_name is None:
        store_name = get_store_name(models, est, runs, n_samples)
            
    results = retrieve_results_dict(store_name)

    if overwrite or (not results):
        printer(1, "No valid, existant results found. Beggining battery.\n")
        results = {}
        for model in models:
            printer(1, "Running: ", model)
            results[model] = run_simulation(
                                runs=runs,
                                n_samples=n_samples,
                                assignment_model=model,
                                estimator=est,
                                verbose=(verbosity==2),
                                **args)
            store_results_dict(results[model], store_name+"_checkpoint_"+model)
            printer(1, "Done.\n")

        store_results_dict(results, store_name)
    else:
        printer(1, "Displaying cached results.\n")
    
    printer(1, "Results")
    for model, results in results.items():
        printer(1, "Model: ", model)
        print(1, "Bias: ", results["Bias"])
        print(1, "RMSE: ", results["RMSE"], "\n")

### Run the Logistic Regression Battery

This one is easy. So we run on one machine.

In [22]:
run_test_battery(
    est=logistic_prop_matching_est,
    runs=1000,
    n_samples=1000)

Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  0.045874914703647685
1 RMSE:  0.07310500057973227 

Model:  B_add_mild_nlin
1 Bias:  3.1844355433209786
1 RMSE:  0.06588422028138122 

Model:  C_add_mod_nlin
1 Bias:  10.094350684204597
1 RMSE:  0.07650839711310455 

Model:  D_mild_nadd_lin
1 Bias:  6.720731771408928
1 RMSE:  0.08531717119502563 

Model:  E_mild_nadd_mild_nlin
1 Bias:  10.36168716658826
1 RMSE:  0.09094245826533698 

Model:  F_mod_nadd_lin
1 Bias:  3.1228082403965436
1 RMSE:  0.07605107262377982 

Model:  G_mod_nadd_mod_nlin
1 Bias:  11.830178367664905
1 RMSE:  0.07798212919046259 



### Run the GenMatch Battery

We split this across three machines using remote clusters.

In [30]:
gm_est = genmatch_est
gm_runs = 1000
gm_n_samples = 1000
gm_models_sets = [assignment_model_names[:3], assignment_model_names[3:5], assignment_model_names[5:]]
gm_files_to_be_produced = []

for model_set in gm_models_sets:
    gm_files_to_be_produced.append(get_store_name(model_set, gm_est, gm_runs, gm_n_samples))

gm_files_to_be_produced

['est_genmatch_est_runs_1000_n_1000_models_A_add_lin_B_add_mild_nlin_C_add_mod_nlin',
 'est_genmatch_est_runs_1000_n_1000_models_D_mild_nadd_lin_E_mild_nadd_mild_nlin',
 'est_genmatch_est_runs_1000_n_1000_models_F_mod_nadd_lin_G_mod_nadd_mod_nlin']

In [31]:
run_test_battery(
    est=gm_est,
    runs=gm_runs,
    n_samples=gm_n_samples,
    models=gm_models_sets[0],
    verbosity=2)

No valid, existant results found. Beggining battery.

Running:  A_add_lin
GenMatch Time:  9.829266786575317
GenMatch Time:  12.110324144363403
GenMatch Time:  11.836768388748169
GenMatch Time:  22.61872124671936
GenMatch Time:  10.672383069992065
GenMatch Time:  19.582879543304443
GenMatch Time:  12.014493227005005
GenMatch Time:  15.595986604690552
GenMatch Time:  11.373863697052002
GenMatch Time:  19.78629207611084
GenMatch Time:  11.653300762176514
GenMatch Time:  11.78925633430481
GenMatch Time:  12.50893259048462
GenMatch Time:  16.401026964187622
GenMatch Time:  27.85374689102173
GenMatch Time:  10.733835697174072
GenMatch Time:  17.674075603485107
GenMatch Time:  12.122838497161865
GenMatch Time:  5.808304786682129
GenMatch Time:  25.942371368408203
GenMatch Time:  14.788551568984985
GenMatch Time:  11.475120544433594
GenMatch Time:  18.243084192276
GenMatch Time:  11.50080919265747
GenMatch Time:  8.017986059188843
GenMatch Time:  14.277287244796753
GenMatch Time:  17.724466085

GenMatch Time:  12.361076354980469
GenMatch Time:  15.20369029045105
GenMatch Time:  10.385145425796509
GenMatch Time:  10.991665124893188
GenMatch Time:  16.031652212142944
GenMatch Time:  14.265010118484497
GenMatch Time:  12.783411741256714
GenMatch Time:  12.5383882522583
GenMatch Time:  13.052167654037476
GenMatch Time:  19.07310724258423
GenMatch Time:  9.911722898483276
GenMatch Time:  11.578041076660156
GenMatch Time:  17.065407037734985
GenMatch Time:  13.530067920684814
GenMatch Time:  12.186645269393921
GenMatch Time:  14.101491928100586
GenMatch Time:  8.143609285354614
GenMatch Time:  11.271634817123413
GenMatch Time:  14.56510591506958
GenMatch Time:  17.303200244903564
GenMatch Time:  15.80841064453125
GenMatch Time:  13.829046726226807
GenMatch Time:  17.83160638809204
GenMatch Time:  11.6909339427948
GenMatch Time:  18.684247732162476
GenMatch Time:  28.831517219543457
GenMatch Time:  9.776340961456299
GenMatch Time:  13.905072689056396
GenMatch Time:  12.6969666481018

GenMatch Time:  12.798903703689575
GenMatch Time:  19.572640419006348
GenMatch Time:  9.52813172340393
GenMatch Time:  10.356467485427856
GenMatch Time:  15.503547430038452
GenMatch Time:  18.148364543914795
GenMatch Time:  18.65658187866211
GenMatch Time:  10.984278202056885
GenMatch Time:  11.344476699829102
GenMatch Time:  11.627498865127563
GenMatch Time:  8.708656072616577
GenMatch Time:  21.05859875679016
GenMatch Time:  19.204986572265625
GenMatch Time:  17.42366933822632
GenMatch Time:  18.902260541915894
GenMatch Time:  12.181703567504883
GenMatch Time:  31.127326011657715
GenMatch Time:  10.202429056167603
GenMatch Time:  13.632068634033203
GenMatch Time:  8.334542989730835
GenMatch Time:  11.179257869720459
GenMatch Time:  9.333480834960938
GenMatch Time:  10.544459819793701
GenMatch Time:  23.043400287628174
GenMatch Time:  18.269813060760498
GenMatch Time:  17.17198133468628
GenMatch Time:  16.13990068435669
GenMatch Time:  19.25630807876587
GenMatch Time:  11.198982238769

GenMatch Time:  13.374872207641602
GenMatch Time:  9.928715705871582
GenMatch Time:  22.750695943832397
GenMatch Time:  21.07864475250244
GenMatch Time:  12.803888320922852
GenMatch Time:  15.032223224639893
GenMatch Time:  13.53502607345581
GenMatch Time:  18.48597526550293
GenMatch Time:  28.94120216369629
GenMatch Time:  29.71375274658203
GenMatch Time:  12.897027492523193
GenMatch Time:  14.361770153045654
GenMatch Time:  15.531460762023926
GenMatch Time:  16.350460290908813
GenMatch Time:  11.310728549957275
GenMatch Time:  18.030134677886963
GenMatch Time:  8.765340089797974
GenMatch Time:  17.63581132888794
GenMatch Time:  15.160022497177124
GenMatch Time:  10.33393120765686
GenMatch Time:  11.537871360778809
GenMatch Time:  10.94707465171814
GenMatch Time:  9.587455034255981
GenMatch Time:  14.78618597984314
GenMatch Time:  21.97925353050232
GenMatch Time:  44.23428988456726
GenMatch Time:  9.831668376922607
GenMatch Time:  18.581125497817993
GenMatch Time:  18.065980672836304


GenMatch Time:  10.928070068359375
GenMatch Time:  22.691431283950806
GenMatch Time:  17.457987308502197
GenMatch Time:  22.00134778022766
GenMatch Time:  18.075303554534912
GenMatch Time:  12.633289337158203
GenMatch Time:  13.028236865997314
GenMatch Time:  19.603753805160522
GenMatch Time:  20.458388328552246
GenMatch Time:  16.86788582801819
GenMatch Time:  11.629210233688354
GenMatch Time:  13.134571075439453
GenMatch Time:  10.35071063041687
GenMatch Time:  15.840717792510986
GenMatch Time:  19.77903413772583
GenMatch Time:  23.684887886047363
GenMatch Time:  15.027809858322144
GenMatch Time:  12.948036432266235
GenMatch Time:  14.007059812545776
GenMatch Time:  13.272461414337158
GenMatch Time:  15.76015853881836
GenMatch Time:  15.900595426559448
GenMatch Time:  9.801846504211426
GenMatch Time:  19.887129545211792
GenMatch Time:  8.814074277877808
GenMatch Time:  14.157444953918457
GenMatch Time:  21.258748292922974
GenMatch Time:  13.40079116821289
GenMatch Time:  10.889741420

...

GenMatch Time:  31.836557149887085
GenMatch Time:  10.499371767044067
GenMatch Time:  13.558139562606812
GenMatch Time:  13.24360704421997
GenMatch Time:  15.346543073654175
GenMatch Time:  23.106337785720825
GenMatch Time:  16.978468894958496
GenMatch Time:  15.284775972366333
GenMatch Time:  8.677541971206665
GenMatch Time:  16.922192335128784
GenMatch Time:  10.382696151733398
GenMatch Time:  10.145409107208252
GenMatch Time:  16.637075424194336
GenMatch Time:  18.396345853805542
Done 900 of 1000
GenMatch Time:  20.05172324180603
GenMatch Time:  17.526580810546875
GenMatch Time:  7.150322437286377
GenMatch Time:  10.737781047821045
GenMatch Time:  10.582868814468384
GenMatch Time:  9.666577100753784
GenMatch Time:  8.453465700149536
GenMatch Time:  15.154508590698242
GenMatch Time:  17.264048099517822
GenMatch Time:  13.830763339996338
GenMatch Time:  10.974889993667603
GenMatch Time:  13.181992769241333
GenMatch Time:  11.491514205932617
GenMatch Time:  29.665709257125854
GenMatch 

GenMatch Time:  15.978049278259277
GenMatch Time:  11.79386281967163
GenMatch Time:  13.600801467895508
GenMatch Time:  17.20580506324768
GenMatch Time:  26.381704330444336
GenMatch Time:  14.675599813461304
GenMatch Time:  12.71271824836731
GenMatch Time:  29.019622802734375
GenMatch Time:  25.18121886253357
GenMatch Time:  14.395731925964355
GenMatch Time:  17.51863431930542
GenMatch Time:  11.028089046478271
GenMatch Time:  16.154726028442383
GenMatch Time:  12.596618890762329
GenMatch Time:  29.43441605567932
GenMatch Time:  16.220956325531006
GenMatch Time:  17.09548330307007
GenMatch Time:  10.057398796081543
GenMatch Time:  16.98680567741394
GenMatch Time:  16.644158124923706
GenMatch Time:  17.008238077163696
GenMatch Time:  10.812776803970337
GenMatch Time:  10.11317491531372
GenMatch Time:  19.573984146118164
GenMatch Time:  10.294697999954224
GenMatch Time:  16.570919275283813
GenMatch Time:  19.82393503189087
GenMatch Time:  9.622871398925781
GenMatch Time:  14.828742027282

GenMatch Time:  10.212096929550171
GenMatch Time:  13.858313083648682
GenMatch Time:  9.696030378341675
GenMatch Time:  16.057172060012817
GenMatch Time:  10.72610092163086
GenMatch Time:  27.221864461898804
GenMatch Time:  19.5240581035614
GenMatch Time:  13.48617672920227
GenMatch Time:  14.067455053329468
GenMatch Time:  24.11187434196472
GenMatch Time:  14.035895109176636
GenMatch Time:  11.469428300857544
GenMatch Time:  21.970168352127075
GenMatch Time:  10.124177694320679
GenMatch Time:  17.25019121170044
GenMatch Time:  10.471933603286743
GenMatch Time:  10.197997331619263
GenMatch Time:  7.015820741653442
GenMatch Time:  10.24987530708313
GenMatch Time:  13.634419441223145
GenMatch Time:  19.07408571243286
GenMatch Time:  11.08762502670288
GenMatch Time:  26.814059495925903
GenMatch Time:  13.893109560012817
GenMatch Time:  13.562575101852417
GenMatch Time:  15.935181617736816
GenMatch Time:  15.364295482635498
GenMatch Time:  12.939769983291626
GenMatch Time:  16.426331996917

GenMatch Time:  8.45350980758667
GenMatch Time:  13.477983951568604
GenMatch Time:  34.93732571601868
GenMatch Time:  9.319049596786499
GenMatch Time:  19.132873058319092
GenMatch Time:  15.261531352996826
GenMatch Time:  24.768421173095703
GenMatch Time:  11.049512386322021
GenMatch Time:  6.087931156158447
Done 600 of 1000
GenMatch Time:  12.437757015228271
GenMatch Time:  18.031426668167114
GenMatch Time:  25.65332269668579
GenMatch Time:  14.707815408706665
GenMatch Time:  15.28645372390747
GenMatch Time:  19.68227505683899
GenMatch Time:  14.551361322402954
GenMatch Time:  26.734496355056763
GenMatch Time:  18.200400829315186
GenMatch Time:  19.652236223220825
GenMatch Time:  13.629003047943115
GenMatch Time:  23.709523677825928
GenMatch Time:  12.596113204956055
GenMatch Time:  20.604854106903076
GenMatch Time:  14.276776552200317
GenMatch Time:  8.979127645492554
GenMatch Time:  31.28819441795349
GenMatch Time:  12.902865648269653
GenMatch Time:  17.561756372451782
GenMatch Time

GenMatch Time:  16.278260946273804
GenMatch Time:  14.266754627227783
GenMatch Time:  12.028504848480225
GenMatch Time:  25.156612157821655
GenMatch Time:  20.935466766357422
GenMatch Time:  8.642917394638062
GenMatch Time:  14.948509693145752
GenMatch Time:  13.378780364990234
GenMatch Time:  13.598713397979736
GenMatch Time:  13.379371881484985
GenMatch Time:  17.41247582435608
GenMatch Time:  15.851682662963867
GenMatch Time:  19.80437731742859
GenMatch Time:  31.94219994544983
GenMatch Time:  19.688245058059692
GenMatch Time:  11.215329647064209
GenMatch Time:  10.554150342941284
GenMatch Time:  12.275270462036133
GenMatch Time:  11.696175575256348
GenMatch Time:  12.168840885162354
GenMatch Time:  8.594035387039185
GenMatch Time:  12.180636167526245
GenMatch Time:  27.581969499588013
GenMatch Time:  12.429024934768677
GenMatch Time:  22.846628665924072
GenMatch Time:  19.382227420806885
GenMatch Time:  42.955785274505615
GenMatch Time:  13.897620439529419
GenMatch Time:  15.031559

In [27]:
run_test_battery(
    est=gm_est,
    runs=gm_runs,
    n_samples=gm_n_samples,
    models=gm_models_sets[1],
    verbosity=2)

Displaying cached results.

Results
Model:  D_mild_nadd_lin
1 Bias:  13.127238676118091
1 RMSE:  0.052508954704472366 

Model:  E_mild_nadd_mild_nlin
1 Bias:  4.4125520199797545
1 RMSE:  0.017650208079919016 



In [28]:
run_test_battery(
    est=gm_est,
    runs=gm_runs,
    n_samples=gm_n_samples,
    models=gm_models_sets[2],
    verbosity=2)

Displaying cached results.

Results
Model:  F_mod_nadd_lin
1 Bias:  17.275217121640445
1 RMSE:  0.06910086848656177 

Model:  G_mod_nadd_mod_nlin
1 Bias:  0.26776452898076564
1 RMSE:  0.0010710581159230625 



#### Combine results

In [40]:
combined_name = get_store_name(assignment_model_names, gm_est, gm_runs, gm_n_samples)

In [41]:
results = {}
for file in gm_files_to_be_produced:
    results.update(retrieve_results_dict(file))

store_results_dict(results, combined_name)
results

{'A_add_lin': {'Bias': 5.5585826624331105, 'RMSE': 0.041571354846349606},
 'B_add_mild_nlin': {'Bias': 4.309919000663494, 'RMSE': 0.03799524129548324},
 'C_add_mod_nlin': {'Bias': 3.715796982487495, 'RMSE': 0.043206587873791204},
 'D_mild_nadd_lin': {'Bias': 2.30672600038697, 'RMSE': 0.040751955269481575},
 'E_mild_nadd_mild_nlin': {'Bias': 1.6356097465092616,
  'RMSE': 0.0388547493767899},
 'F_mod_nadd_lin': {'Bias': 5.058677376450292, 'RMSE': 0.04404046330729526},
 'G_mod_nadd_mod_nlin': {'Bias': 3.185538423779952,
  'RMSE': 0.04439998296725508}}